In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

Let's import all we are going to need

In [61]:
import streamlit as st
import pandas as pd
import numpy as np
from PIL import Image
import requests
import plotly.graph_objects as go
import plotly.express as px
import ccxt
import os
from dotenv import load_dotenv
from datetime import date, datetime
import time
import calendar
from plotly.subplots import make_subplots

from dashboard.static import *

pd.set_option("display.max_columns", None)  # To show all columns

<IPython.core.display.Javascript object>

Les's use the API

In [3]:
api_url = "https://ftx.com/api"

<IPython.core.display.Javascript object>

Let's connect with the API using CCXT

In [4]:
# read .env file
load_dotenv()

# connect exchange with ccxt
exchange_id = "ftx"
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class(
    {
        "enableRateLimit": True,
        "apiKey": os.environ.get("APIKey"),
        "secret": os.environ.get("APISecret"),
    }
)

<IPython.core.display.Javascript object>

Let's select only some coins/USD

In [5]:
markets = requests.get("https://ftx.com/api/markets").json()
markets_df = pd.DataFrame(markets["result"]).set_index("name")
markets_df.loc[symbol_list]

,enabled,postOnly,priceIncrement,sizeIncrement,minProvideSize,last,bid,ask,price,type,futureType,baseCurrency,isEtfMarket,quoteCurrency,underlying,restricted,highLeverageFeeExempt,largeOrderThreshold,change1h,change24h,changeBod,quoteVolume24h,volumeUsd24h,priceHigh24h,priceLow24h,tokenizedEquity
name,,,,,,,,,,,,,,,,,,,,,,,,,,
BTC/USD,True,False,1.000000,0.0001,0.0001,19398.00000,19397.0000,19398.00000,19398.0000,spot,None,BTC,False,USD,None,False,True,2400.0,0.001239,-0.010508,-0.002160,7.714859e+08,7.714859e+08,19767.000000,18847.000000,NaN
ETH/USD,True,False,0.100000,0.0010,0.0010,1334.30000,1334.4000,1334.50000,1334.4000,spot,None,ETH,False,USD,None,False,True,2400.0,-0.000300,-0.001795,-0.005070,5.623911e+08,5.623911e+08,1355.300000,1288.500000,NaN
XRP/USD,True,False,0.000025,1.0000,1.0000,0.43735,0.4374,0.43745,0.4374,spot,None,XRP,False,USD,None,False,True,2400.0,0.002694,-0.004382,-0.029833,7.387540e+07,7.387540e+07,0.455775,0.426275,NaN
SOL/USD,True,False,0.002500,0.0100,0.0100,33.83250,33.8275,33.83000,33.8300,spot,None,SOL,False,USD,None,False,True,2000.0,-0.000443,0.005051,0.015382,5.719500e+07,5.719500e+07,34.430000,32.622500,NaN
USDT/USD,True,False,0.000100,0.0100,0.0100,1.00010,1.0000,1.00010,1.0001,spot,None,USDT,False,USD,None,False,True,2400.0,0.000000,0.000200,0.000200,7.898468e+07,7.898468e+07,1.000100,0.999800,NaN
ETHW/USD,True,False,0.001000,0.0010,0.0010,12.20900,12.2070,12.21900,12.2090,spot,None,ETHW,False,USD,None,False,True,280.0,0.021503,0.118655,0.129104,2.812470e+07,2.812470e+07,12.950000,10.339000,NaN
BNB/USD,True,False,0.001000,0.0100,0.0100,282.84700,282.8210,282.85300,282.8470,spot,None,BNB,False,USD,None,False,True,2000.0,0.001395,0.009879,0.000917,1.851012e+07,1.851012e+07,285.541000,277.169000,NaN
LINK/USD,True,False,0.000500,0.1000,0.1000,7.83200,7.8295,7.83300,7.8320,spot,None,LINK,False,USD,None,False,True,2000.0,0.003138,-0.013167,0.008239,1.749140e+07,1.749140e+07,7.983000,7.574500,NaN
FTT/USD,True,False,0.001000,0.1000,0.1000,24.13300,24.1350,24.14100,24.1350,spot,None,FTT,False,USD,None,False,True,280.0,0.001535,0.001452,-0.000041,1.344939e+07,1.344939e+07,24.353000,23.510000,NaN


<IPython.core.display.Javascript object>

In [6]:
markets = exchange.load_markets()
markets_df = pd.DataFrame(markets).T.set_index("id")
markets_df.loc[symbol_list]

,symbol,base,quote,baseId,quoteId,active,type,linear,inverse,spot,swap,future,option,margin,contract,contractSize,expiry,expiryDatetime,optionType,strike,settle,settleId,precision,limits,info,percentage,tierBased,maker,taker,tiers
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BTC/USD,BTC/USD,BTC,USD,BTC,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.0001, 'price': 1.0}","{'amount': {'min': 0.0001, 'max': None}, 'pric...","{'name': 'BTC/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
ETH/USD,ETH/USD,ETH,USD,ETH,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.001, 'price': 0.1}","{'amount': {'min': 0.001, 'max': None}, 'price...","{'name': 'ETH/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
XRP/USD,XRP/USD,XRP,USD,XRP,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 1.0, 'price': 2.5e-05}","{'amount': {'min': 1.0, 'max': None}, 'price':...","{'name': 'XRP/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
SOL/USD,SOL/USD,SOL,USD,SOL,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.01, 'price': 0.0025}","{'amount': {'min': 0.01, 'max': None}, 'price'...","{'name': 'SOL/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
USDT/USD,USDT/USD,USDT,USD,USDT,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.01, 'price': 0.0001}","{'amount': {'min': 0.01, 'max': None}, 'price'...","{'name': 'USDT/USD', 'enabled': True, 'postOnl...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
ETHW/USD,ETHW/USD,ETHW,USD,ETHW,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.001, 'price': 0.001}","{'amount': {'min': 0.001, 'max': None}, 'price...","{'name': 'ETHW/USD', 'enabled': True, 'postOnl...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
BNB/USD,BNB/USD,BNB,USD,BNB,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.01, 'price': 0.001}","{'amount': {'min': 0.01, 'max': None}, 'price'...","{'name': 'BNB/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
LINK/USD,LINK/USD,LINK,USD,LINK,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.1, 'price': 0.0005}","{'amount': {'min': 0.1, 'max': None}, 'price':...","{'name': 'LINK/USD', 'enabled': True, 'postOnl...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."
FTT/USD,FTT/USD,FTT,USD,FTT,USD,True,spot,None,None,True,False,False,False,True,False,1.0,None,None,None,None,None,None,"{'amount': 0.1, 'price': 0.001}","{'amount': {'min': 0.1, 'max': None}, 'price':...","{'name': 'FTT/USD', 'enabled': True, 'postOnly...",True,True,0.0002,0.0007,"{'taker': [[0.0, 0.0007], [2000000.0, 0.0006],..."


<IPython.core.display.Javascript object>

OHLCV get historical price, volume and trading data for a given base asset, quote asset, exchange, instrument type and resolution interval for this time period

In [7]:
ohlcv_s = {}
if exchange.has["fetchOHLCV"]:
    for symbol in exchange.markets:
        if symbol in symbol_list:
            ohlcv_s[symbol] = pd.DataFrame(
                exchange.fetch_ohlcv(symbol, "1d", limit=100000),
                columns=["date", "open", "high", "low", "close", "volume"],
            )

<IPython.core.display.Javascript object>

In [8]:
for ohlcv in ohlcv_s:
    ohlcv_s[ohlcv]["date"] = pd.to_datetime(ohlcv_s[ohlcv]["date"], unit="ms")
    print(ohlcv, end="\n")
    print(ohlcv_s[ohlcv].tail(5))
    print(ohlcv_s[ohlcv].describe())
    print("-----------", end="\n")

ATOM/USD
          date     open     high     low    close        volume
208 2022-09-25  13.6350  14.8425  13.590  13.9800  7.278558e+06
209 2022-09-26  13.9800  14.8925  13.755  13.9100  1.382479e+07
210 2022-09-27  13.9100  14.7400  13.135  13.4825  1.308799e+07
211 2022-09-28  13.4825  13.5800  12.620  12.9925  1.579316e+07
212 2022-09-29  12.9925  13.9500  12.425  12.6025  6.965600e+06
             open        high         low       close        volume
count  213.000000  213.000000  213.000000  213.000000  2.130000e+02
mean    15.658398   16.343885   14.946162   15.565657  8.047890e+06
std      7.866382    8.047056    7.628608    7.784610  5.908707e+06
min      6.067500    6.505000    5.550000    6.067500  6.170731e+05
25%      9.627500   10.330000    9.142500    9.627500  3.575193e+06
50%     11.910000   12.560000   11.435000   11.910000  6.503322e+06
75%     23.212500   23.492500   22.450000   23.085000  1.098228e+07
max     34.080000   35.275000   31.150000   34.080000  2.862224

XRP/USD
          date      open      high       low     close        volume
798 2022-09-25  0.487900  0.520925  0.475725  0.491600  9.570244e+07
799 2022-09-26  0.491575  0.501875  0.457625  0.468200  9.158187e+07
800 2022-09-27  0.468200  0.487775  0.434875  0.445875  9.873863e+07
801 2022-09-28  0.445875  0.455775  0.416650  0.449025  8.335614e+07
802 2022-09-29  0.449025  0.451750  0.426050  0.437475  6.273917e+07
             open        high         low       close        volume
count  803.000000  803.000000  803.000000  803.000000  8.030000e+02
mean     0.644267    0.674963    0.610225    0.644566  2.604983e+07
std      0.346529    0.367313    0.322908    0.346242  2.656061e+07
min      0.194575    0.200050    0.173100    0.194500  8.106712e+03
25%      0.335812    0.348275    0.326313    0.336487  2.388543e+06
50%      0.597475    0.623750    0.561200    0.597475  2.214970e+07
75%      0.851988    0.885563    0.819900    0.851988  3.887495e+07
max      1.834900    1.968575    1

<IPython.core.display.Javascript object>

A ticker is simply an indicator of a certain asset that is regularly updated by the different stock market exchanges throughout the trading session. Those who are familiar with the content of a financial news channel have probably seen the bottom of the display's unending loop of stocks and commodities scrolling by at a tremendous pace.

In [9]:
tickers = pd.DataFrame(exchange.fetch_tickers(symbols=symbol_list)).T.set_index(
    "symbol"
)

<IPython.core.display.Javascript object>

In [10]:
tickers[["percentage", "quoteVolume"]]

,percentage,quoteVolume
symbol,,
ATOM/USD,-3.33909,7947410.46
BNB/USD,0.975436,18510116.86253
BTC/USD,-1.040604,771485932.6275
ETH/USD,-0.157092,562391138.5384
ETHW/USD,11.902144,28124700.433726
FTT/USD,0.170124,13449393.3988
LINK/USD,-1.291501,17491402.9245
SOL/USD,0.564468,57195000.4609
USDT/USD,0.020002,78984678.200175


<IPython.core.display.Javascript object>

In [11]:
URL_BASE = "https://ftx.com/api"


def get_market():
    url = f"{URL_BASE}/markets/BTC/USD"
    res = requests.get(url).json()
    result = pd.DataFrame(res["result"], index=[0])
    priceHigh24h = float(result["priceHigh24h"])
    priceLow24h = float(result["priceLow24h"])
    volumeUsd24h = float(result["volumeUsd24h"])
    change24h = float(result["change24h"])
    return priceHigh24h, priceLow24h, volumeUsd24h, change24h

<IPython.core.display.Javascript object>

In [12]:
get_market()

(19767.0, 18847.0, 771485932.6275, -0.009947457021884406)

<IPython.core.display.Javascript object>

In [49]:
n = pd.DataFrame(exchange.markets["BTC/USD"])
price = n["info"]["price"]
change1h = n["info"]["change1h"]
change24h = n["info"]["change24h"]
changeBod = n["info"]["changeBod"]
quoteVolume24h = n["info"]["quoteVolume24h"]
volumeUsd24h = n["info"]["volumeUsd24h"]
priceHigh24h = n["info"]["priceHigh24h"]
priceLow24h = n["info"]["priceLow24h"]
print(
    price,
    change1h,
    change24h,
    changeBod,
    quoteVolume24h,
    volumeUsd24h,
    priceHigh24h,
    priceLow24h,
)

19398.0 0.0012387736141220192 -0.010508059579677617 -0.0021604938271604936 771485932.6275 771485932.6275 19767.0 18847.0


<IPython.core.display.Javascript object>

In [40]:
n

'19398.0'

<IPython.core.display.Javascript object>

In [48]:
n["info"]

amount                                      NaN
price                                   19398.0
cost                                        NaN
leverage                                    NaN
name                                    BTC/USD
enabled                                    True
postOnly                                  False
priceIncrement                              1.0
sizeIncrement                            0.0001
minProvideSize                           0.0001
last                                    19398.0
bid                                     19397.0
ask                                     19398.0
type                                       spot
futureType                                 None
baseCurrency                                BTC
isEtfMarket                               False
quoteCurrency                               USD
underlying                                 None
restricted                                False
highLeverageFeeExempt                   

<IPython.core.display.Javascript object>

In [17]:
p = exchange.timeframes

<IPython.core.display.Javascript object>

In [25]:
[x for x in p.keys()]

['15s', '1m', '5m', '15m', '1h', '4h', '1d', '3d', '1w', '2w', '1M']

<IPython.core.display.Javascript object>

In [71]:
def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple()) * 1000
    ohlcv1 = exchange.fetch_ohlcv(symbol=pair, timeframe="1m", since=since, limit=limit)
    ohlcv2 = exchange.fetch_ohlcv(symbol=pair, timeframe="1m", since=since, limit=limit)
    ohlcv = ohlcv1 + ohlcv2
    return ohlcv


def ohlcv(dt, pair, period="1d"):
    ohlcv = []
    limit = 1000
    if period == "1m":
        limit = 720
    elif period == "1d":
        limit = 365
    elif period == "1h":
        limit = 24
    elif period == "5m":
        limit = 288
    for i in dt:
        start_dt = datetime.strptime(i, "%Y%m%d")
        since = calendar.timegm(start_dt.utctimetuple()) * 1000
        if period == "1m":
            ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        else:
            ohlcv.extend(
                exchange.fetch_ohlcv(
                    symbol=pair, timeframe=period, since=since, limit=limit
                )
            )
    df = pd.DataFrame(ohlcv, columns=["Time", "Open", "High", "Low", "Close", "Volume"])
    df["Time"] = [datetime.fromtimestamp(float(time) / 1000) for time in df["Time"]]
    df["Open"] = df["Open"].astype(np.float64)
    df["High"] = df["High"].astype(np.float64)
    df["Low"] = df["Low"].astype(np.float64)
    df["Close"] = df["Close"].astype(np.float64)
    df["Volume"] = df["Volume"].astype(np.float64)
    # df.set_index("Time", inplace=True)
    return df

<IPython.core.display.Javascript object>

In [73]:
dt = ["20220101", "20220929"]
df = ohlcv(dt, "BTC/USD", period="1M")
df

,Time,Open,High,Low,Close,Volume
0,2021-12-31 19:00:00,46199.0,48255.0,32915.0,37909.0,1.590282e+10
1,2022-01-30 19:00:00,37909.0,45880.0,34334.0,44452.0,1.584033e+10
2,2022-03-01 19:00:00,44452.0,48224.0,37154.0,45523.0,1.447041e+10
3,2022-03-31 19:00:00,45523.0,47462.0,37574.0,37630.0,1.200522e+10
4,2022-04-30 19:00:00,37630.0,40026.0,25429.0,31711.0,2.421718e+10
5,2022-05-30 19:00:00,31711.0,32380.0,17580.0,20096.0,2.617592e+10
6,2022-06-29 19:00:00,20096.0,24452.0,18611.0,23784.0,1.815205e+10
7,2022-07-29 19:00:00,23784.0,25215.0,19516.0,19555.0,1.410498e+10
8,2022-08-28 19:00:00,19555.0,22912.0,18142.0,19077.0,1.634034e+10
9,2022-09-27 19:00:00,19078.0,19769.0,18464.0,19532.0,1.622716e+09


<IPython.core.display.Javascript object>

In [74]:
print(date(2022, 1, 1).strftime("%Y%m%d"), datetime.now().strftime("%Y%m%d"))

20220101 20220929


<IPython.core.display.Javascript object>

In [122]:
start_date = date(2021, 1, 1).strftime("%Y%m%d")
end_date = datetime.now().strftime("%Y%m%d")
dt = [start_date, end_date]
df = ohlcv(dt, "BTC/USD", "1M")
df

,Time,Open,High,Low,Close,Volume
0,2020-12-31 19:00:00,28965.0,42000.0,27700.5,34299.0,4.762470e+09
1,2021-01-30 19:00:00,34299.0,58366.0,32190.0,49631.0,6.479471e+09
2,2021-03-01 19:00:00,49631.0,61806.0,46281.0,58790.0,8.143631e+09
3,2021-03-31 19:00:00,58790.0,64887.0,47000.0,57770.0,9.037272e+09
4,2021-04-30 19:00:00,57770.0,59643.0,29150.0,35660.0,1.673837e+10
5,2021-05-30 19:00:00,35660.0,41347.0,28819.0,35919.0,1.483687e+10
6,2021-06-29 19:00:00,35919.0,40937.0,29284.0,40046.0,1.310000e+10
7,2021-07-29 19:00:00,40046.0,50549.0,37258.0,48937.0,2.020189e+10
8,2021-08-28 19:00:00,48937.0,52961.0,39710.0,42154.0,1.599203e+10
9,2021-09-27 19:00:00,42153.0,67082.0,40751.0,58449.0,1.937174e+10


<IPython.core.display.Javascript object>

In [88]:
variance = round(np.var(df["Close"]), 2)
variance

169317353.17

<IPython.core.display.Javascript object>

In [91]:
df["RAC"] = df["Close"].rolling(window=5, center=True).mean()

<IPython.core.display.Javascript object>

In [92]:
df

,Time,Open,High,Low,Close,Volume,RAC
0,2020-12-31 19:00:00,28965.0,42000.0,27700.5,34299.0,4.762470e+09,NaN
1,2021-01-30 19:00:00,34299.0,58366.0,32190.0,49631.0,6.479471e+09,NaN
2,2021-03-01 19:00:00,49631.0,61806.0,46281.0,58790.0,8.143631e+09,47230.0
3,2021-03-31 19:00:00,58790.0,64887.0,47000.0,57770.0,9.037272e+09,47554.0
4,2021-04-30 19:00:00,57770.0,59643.0,29150.0,35660.0,1.673837e+10,45637.0
5,2021-05-30 19:00:00,35660.0,41347.0,28819.0,35919.0,1.483687e+10,43666.4
6,2021-06-29 19:00:00,35919.0,40937.0,29284.0,40046.0,1.310000e+10,40543.2
7,2021-07-29 19:00:00,40046.0,50549.0,37258.0,48937.0,2.020189e+10,45101.0
8,2021-08-28 19:00:00,48937.0,52961.0,39710.0,42154.0,1.599203e+10,48666.6
9,2021-09-27 19:00:00,42153.0,67082.0,40751.0,58449.0,1.937174e+10,50815.8


<IPython.core.display.Javascript object>

In [120]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[200, 50]
)
fig.add_trace(
    go.Candlestick(
        x=df["Time"],
        open=df["Open"],
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        name="Candlestick",
    ),
    row=1,
    col=1,
)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.add_trace(
    go.Scatter(
        x=df["Time"],
        y=df["RAC"],
        line=dict(color="black", width=2, dash="dot"),
        name="RAC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Bar(
        x=df["Time"],
        y=df["Volume"],
        marker=dict(color=df["Volume"], colorscale="Cividis_r"),
        name="Volumen",
    ),
    row=2,
    col=1,
)
fig["layout"]["xaxis2"]["title"] = "Time"
fig["layout"]["yaxis"]["title"] = "Price"
fig["layout"]["yaxis2"]["title"] = "Volume"
fig.show()

<IPython.core.display.Javascript object>